In [ ]:
import numpy as np
from Base_Connectanator import Base_Connectanator
import pygame as pg

CONNECT_NUM = 4

pygame 2.6.1 (SDL 2.28.4, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Base_Connectanator():
    def __init__(self, connect_num:int=4):
        self.connect_num = connect_num
        self.slots = 2 * connect_num -1
        self.rows = self.slots - 1
        self.moves = range(self.slots)
        self.players = ("Player 1", "Player 2") # ("Red", "Yellow")
        self.board = np.zeros((self.rows, self.slots), dtype=int)
        self.turn = 0

    def players_move(self, move):
        try: 
            move = int(move)
        except ValueError:
            return None
        
        if move not in self.moves:
            return None
        elif self.board[0, move] != 0:
            return None
        else: 
            return move


    def place_counter(self):
        col = self.board[:, self.move]
        for i, val in enumerate(col):
            if val != 0:
                placement = (i-1, self.move)
                break
            elif i == self.rows-1:
                placement = (i, self.move)

        self.board[placement] = self.get_player()
        return placement


    def game_turn(self, players_move=None):
        if players_move is None:
            return False, self.get_player()
        
        self.set_move(players_move)
        placement = self.place_counter()
        winner = self.any_winners(placement)
        if not winner:
            self.turn += 1
        return winner, self.get_player()

    def any_winners(self, placement):
        # think there might be a better way to return the value of this 
        # and defo need to rewite the detection cause 'tis fugly
        o = self.get_player()
        row, col = placement

        # checking rows
        connected = 0
        for i in range(self.rows):
            if self.board[i, col] == o:
                connected += 1
                if connected == self.connect_num:
                    return True
            else: 
                connected = 0

        # checking cols
        connected = 0
        for i in range(self.slots):
            if self.board[row, i] == o:
                connected += 1
                if connected == self.connect_num:
                    return True
            else: 
                connected = 0
    
        # checking down-sloppiong diagonal
        connected = 0
        if col > row:
            c = col - row
            r = 0
        else:
            r = row - col
            c = 0
        for i in range(self.rows):
            if self.board[r, c] == o:
                connected += 1
                if connected == self.connect_num:
                    return True
            else:
                connected = 0

            r += 1; c += 1
            if r == self.rows or c == self.slots:
                break
            
        # checking upwards diagonals 
        connected = 0
        R = self.rows - 1
        if col + row > R:
            r = R
            c = col - (R - row)
        else:
            c = 0
            r = row + col
        for i in range(self.rows):
            if self.board[r, c] == o:
                connected += 1
                if connected == self.connect_num:
                    return True
            else:
                connected = 0

            r -= 1; c += 1
            if r == -1 or c == self.slots:
                break
        return False

    def get_player_name(self):
        return self.players[self.get_player()-1]
    
    def get_turn(self):
        return self.turn
    
    def get_player(self):
        return self.turn % 2 + 1
    
    def get_board(self):
        return self.board
    
    def set_move(self, move):
        self.move = move

In [16]:
class Gameplay_Happenater():
    def __init__(
            self,
            connect_num:int,
            res=(1280,720),
            frame_rate=60,
        ):
        pg.init()
        pg.key.set_repeat(10)
        self.connect_num = connect_num
        self.res = res
        self.screen = pg.display.set_mode(res)
        self.frame_rate = frame_rate
        self.clock = pg.time.Clock()
        self.running = False

        self.slots = 2 * connect_num - 1
        self.rows = self.slots - 1
        self.slot_px = int(res[0] / self.slots)
        self.row_px = int(res[1] / self.rows)
        # making grid of circle centers:
        self.grid = [
            (self.slot_px*(i+0.5), self.row_px*(j+0.5)) 
                for i in range(self.slots) 
                for j in range(self.rows)
        ]
        self.cir_rad = int(min([self.row_px, self.slot_px])/2 - 5)
        self.board = np.zeros((self.rows, self.slots), dtype=int)
        self.curr_slot = 0


    def game_runner(self, board):
        self.set_board(board)
        move = self.handle_inputs()
        self.display_func()
        return move


    def handle_inputs(self):
        for event in pg.event.get():
            if event.type == pg.QUIT:
                self.set_running(False)
        
        keys = pg.key.get_pressed()
        if keys[pg.K_ESCAPE]:
            self.set_running(False)
        if True: #keys[pg.K_LEFT]:
            self.curr_slot = int((self.curr_slot - 1) % self.slots)
        if keys[pg.K_RIGHT]:
            self.curr_slot = int((self.curr_slot + 1) % self.slots)
        if keys[pg.K_RETURN]:
            return self.curr_slot
        return None


    def display_func(self):
        self.screen.fill((0,0,150))
        rect_left = int(self.grid[self.curr_slot][0] - 0.5*self.slot_px)
        self.screen.fill(
            color="Yellow",
            rect=(rect_left, 0, 2*self.cir_rad, self.res[1])
        )
        colors = ["Black", "Red", "Yellow"]
        for n, point in enumerate(self.grid):
            c = colors[self.board.flatten()[n]]
            pg.draw.circle(
                surface=self.screen,
                color=c,
                center=point,
                radius=self.cir_rad
            )
        pg.display.flip()
        self.clock.tick(self.frame_rate)
    

    def set_running(self, val:bool):
        self.running = val
    
    def set_board(self, board):
        self.board = board
    

In [18]:
def main():
    fe = Gameplay_Happenater(CONNECT_NUM)
    be = Base_Connectanator(CONNECT_NUM)

    fe.set_running(True)
    while fe.running:
        try:
            board = be.get_board()
            move = fe.game_runner(board)
            any_win, player_turn = be.game_turn(move)
            if any_win:
                print(f'Player {player_turn} Wins!!!')
        except Exception as e:
            pg.quit()
            raise e
    pg.quit()


if __name__ == "__main__":
    main()

KeyboardInterrupt: 